<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20Underwater%20Fish%20Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## 1. Installation Instructions



## 2. How to train a custom fish segmentation algorithm

# About the networks


1. UNet
  - https://arxiv.org/abs/1505.04597
  - https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
  - https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5


2. FPN
  - http://openaccess.thecvf.com/content_cvpr_2017/papers/Lin_Feature_Pyramid_Networks_CVPR_2017_paper.pdf
  - https://towardsdatascience.com/review-fpn-feature-pyramid-network-object-detection-262fc7482610
  - https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c


3. PSPNet
  - https://arxiv.org/abs/1612.01105
  - https://towardsdatascience.com/review-pspnet-winner-in-ilsvrc-2016-semantic-segmentation-scene-parsing-e089e5df177d
  - https://developers.arcgis.com/python/guide/how-pspnet-works/


4. Linknet
  - https://arxiv.org/pdf/1707.03718.pdf
  - https://neptune.ai/blog/image-segmentation-tips-and-tricks-from-kaggle-competitions

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/9_segmentation_models/installation
     
 - Select the right requirements file and run
 
     - cat requirements_cuda9.0.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install


# For Local systems and cloud select the right CUDA version
#! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_cuda10.0.txt | xargs -n 1 -L 1 pip install

# Train you own detector

# Monk Format

## Dataset Directory Structure

      root_dir
          |
          | 
          |         
          |----train_img_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on) 
          |
          |----train_mask_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_img_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_mask_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)

          
          

# Sample Dataset Credits

    credits: http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/

In [ ]:
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_01.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_02.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_03.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_04.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_05.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_06.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_07.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_08.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_09.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_10.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_11.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_12.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_13.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_14.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_15.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_16.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_17.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_18.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_19.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_20.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_21.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_22.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/fish_image/fish_23.tar

In [ ]:
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_01.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_02.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_03.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_04.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_05.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_06.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_07.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_08.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_09.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_10.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_11.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_12.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_13.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_14.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_15.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_16.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_17.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_18.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_19.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_20.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_21.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_22.tar
! wget http://groups.inf.ed.ac.uk/f4k/GROUNDTRUTH/RECOG/Archive/mask_image/mask_23.tar

In [ ]:
! tar -xf fish_01.tar
! tar -xf fish_02.tar
! tar -xf fish_03.tar
! tar -xf fish_04.tar
! tar -xf fish_05.tar
! tar -xf fish_06.tar
! tar -xf fish_07.tar
! tar -xf fish_08.tar
! tar -xf fish_09.tar
! tar -xf fish_10.tar
! tar -xf fish_11.tar
! tar -xf fish_12.tar
! tar -xf fish_13.tar
! tar -xf fish_14.tar
! tar -xf fish_15.tar
! tar -xf fish_16.tar
! tar -xf fish_17.tar
! tar -xf fish_18.tar
! tar -xf fish_19.tar
! tar -xf fish_20.tar
! tar -xf fish_21.tar
! tar -xf fish_22.tar
! tar -xf fish_23.tar

In [ ]:
! tar -xf mask_01.tar
! tar -xf mask_02.tar
! tar -xf mask_03.tar
! tar -xf mask_04.tar
! tar -xf mask_05.tar
! tar -xf mask_06.tar
! tar -xf mask_07.tar
! tar -xf mask_08.tar
! tar -xf mask_09.tar
! tar -xf mask_10.tar
! tar -xf mask_11.tar
! tar -xf mask_12.tar
! tar -xf mask_13.tar
! tar -xf mask_14.tar
! tar -xf mask_15.tar
! tar -xf mask_16.tar
! tar -xf mask_17.tar
! tar -xf mask_18.tar
! tar -xf mask_19.tar
! tar -xf mask_20.tar
! tar -xf mask_21.tar
! tar -xf mask_22.tar
! tar -xf mask_22.tar
! tar -xf mask_23.tar

In [ ]:
! mkdir dataset
! mkdir dataset/images
! mkdir dataset/masks

In [ ]:
import os
import cv2
import numpy as np 
from tqdm import tqdm

In [ ]:
num = 0;
for i in tqdm(range(1, 24)):
    if(i < 10):
        img_folder = "fish_0" + str(i)
        mask_folder = "mask_0" + str(i);
    else:
        img_folder = "fish_" + str(i)
        mask_folder = "mask_" + str(i);

    img_list = sorted(os.listdir(img_folder))
    mask_list = sorted(os.listdir(mask_folder))  

    for j in tqdm(range(len(img_list))):
        img = cv2.imread(img_folder + "/" + img_list[j]);
        mask = cv2.imread(mask_folder + "/" + mask_list[j]);


        mask[mask > 0] = 1;

        cv2.imwrite("dataset/images/" + str(num) + ".png", img);
        cv2.imwrite("dataset/masks/" + str(num) + ".png", mask);

        num += 1;

## Training 

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from train_segmentation import Segmenter

In [ ]:
gtf = Segmenter();

In [ ]:
train_img_dir = "dataset/images";
train_mask_dir = "dataset/masks";

val_img_dir = "dataset/images";
val_mask_dir = "dataset/masks";


In [ ]:
classes_dict = {
    'background': 0, 
    'fish': 1
};
classes_to_train = ['background', 'fish'];

In [ ]:
gtf.Train_Dataset(train_img_dir, train_mask_dir, classes_dict, classes_to_train)

In [ ]:
gtf.Val_Dataset(val_img_dir, val_mask_dir)

In [ ]:
gtf.List_Backbones();

In [ ]:
gtf.Data_Params(batch_size=2, backbone="efficientnetb3", image_shape=[103, 73])

In [ ]:
gtf.List_Models();

In [ ]:
gtf.Model_Params(model="Unet")

In [ ]:
gtf.Train_Params(lr=0.0001)

In [ ]:
gtf.Setup();

In [ ]:
gtf.Train(num_epochs=300);

In [ ]:
gtf.Visualize_Training_History();

# Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0, 
    'fish': 1
};
classes_to_train = ['background', 'fish'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[103, 73])

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
import os
img_list = os.listdir("dataset/images/")

In [ ]:
gtf.Predict("dataset/images/" + img_list[i], vis=True);